In [ ]:
import pathlib
import sys
from gymnasium import spaces

sys.path.append(str(pathlib.Path().absolute().parent))

from src.replay_memory import FastReplayBuffer
from src.env import FourRoomEnvWithTagging


In [ ]:
# m = FastReplayBuffer(max_size=5, state_size=1, trajectory_size=3)

# m.add([0], 1, 1, [2], 0, 1) # 0
# m.add([1], 2, 2, [3], 0, 1) # 1
# m.add([2], 3, 3, [4], 0, 0) # 2
# m.add([3], 4, 4, [5], 0, 0) # 3
# m.add([4], 4, 4, [5], 0, 0) # 4
# m.add([5], 4, 4, [5], 0, 0) # 5
# m.add([6], 4, 4, [5], 0, 1) # 6

# m.sample(5).states

In [ ]:
BUF_SIZE = 3000
env = FourRoomEnvWithTagging(n_imposters=1, n_crew=4, n_jobs=5)
m = FastReplayBuffer(max_size=BUF_SIZE, state_size=env.flattened_state_size, trajectory_size=3, n_agents=env.n_agents)
m.populate(env, 1000)

In [ ]:
batch = m.sample(5)

In [ ]:
batch.agent_order

In [ ]:
m.starts.sum()

In [ ]:
m.dones.unique(return_counts=True)

In [ ]:
m.added